# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902011


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:05<01:28,  2.94s/it]

Rendering models:   9%|▉         | 3/32 [00:05<01:00,  2.09s/it]

Rendering models:  12%|█▎        | 4/32 [00:07<00:50,  1.80s/it]

Rendering models:  16%|█▌        | 5/32 [00:07<00:37,  1.40s/it]

...

Rendering models:  91%|█████████ | 29/32 [00:14<00:00,  5.75it/s]

Rendering models:  94%|█████████▍| 30/32 [00:14<00:00,  3.96it/s]

Rendering models:  97%|█████████▋| 31/32 [00:14<00:00,  4.14it/s]

Rendering models: 100%|██████████| 32/32 [00:14<00:00,  4.42it/s]

equidad1                               0.095079
taisribeiro                            0.012218
rjsmethurst                            0.000500
Lupinki                                0.001457
Jes8x                                  0.001609
cmk24                                  0.000284
not-logged-in-0af0e477ffde39f6315d     0.002504
bronzesword                            0.000656
not-logged-in-2e775991bd3ca814ca1d     0.007762
Joannasephine                          0.001929
tomburgerpie                           0.003004
ElisabethB                             0.031141
Petrusperes                            0.021635
redboy21                               0.492930
sn346808                               0.028490
not-logged-in-14d09f70263c1fe15cc3     0.000335
not-logged-in-d92d5297031a7da36b6d     0.000464
not-logged-in-2c779a6772b8571dd609     1.981485
Frosst.matt                            0.001474
skotecha                               0.000473
not-logged-in-2eb38841187e4efc56ae     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())